In [1]:
import json
import pandas as pd
import numpy as np
import datetime
from airflow.providers.postgres.hooks.postgres import PostgresHook
from airflow.providers.mysql.hooks.mysql import MySqlHook
from fuzzywuzzy import fuzz
import seaborn as sns

warehouse_hook = PostgresHook(postgres_conn_id='airflow_warehouse_user', schema='ubuntu')
mifos_hook = MySqlHook(mysql_conn_id='mifos_db', schema='mifostenant-safaricom')
local_airflow_hook_hook = MySqlHook(mysql_conn_id='local_airflow_db', schema='airflow_db')

/tmp/ipykernel_17801/100039020.py:10 DeprecationWarning: The "schema" arg has been renamed to "database" as it contained the database name.Please use "database" to set the database name.

In [3]:
dt = pd.read_excel('~/data/ORG_4018601_Loan Repayment Account_Completed_20230221103517 (1).xls', skiprows=6)
dt2 = pd.read_excel('~/data/ORG_4018601_Loan Disbursement Account_Completed_20230221103434 (1).xls', skiprows=6)

In [6]:
existing_disb = warehouse_hook.get_pandas_df(
    sql="""
        select fund_movement_transaction_id from bloomlive.raw_tep_disbursements where fund_movement_transaction_id in %(fmti)s
    """,
    parameters={'fmti': tuple(dt2['Receipt No.'].tolist())}
)
missing_disb = dt2[~dt2['Receipt No.'].isin(existing_disb['fund_movement_transaction_id'].tolist())]
existing_rep = warehouse_hook.get_pandas_df(
    sql="""
        select fund_movement_transaction_id from bloomlive.raw_tep_repayments where fund_movement_transaction_id in %(fmti)s
    """,
    parameters={'fmti': tuple(dt['Receipt No.'].tolist())}
)
missing_rep = dt[~dt['Receipt No.'].isin(existing_rep['fund_movement_transaction_id'].tolist())]

[2023-06-12 16:29:04,791] {base.py:73} INFO - Using connection ID 'airflow_warehouse_user' for task execution.


OperationalError: connection to server at "157.245.248.249", port 5432 failed: Connection timed out
	Is the server running on that host and accepting TCP/IP connections?


In [23]:
missing_disb.drop_duplicates(subset=['Receipt No.'])

,Receipt No.,Completion Time,Initiation Time,Details,Transaction Status,Paid In,Withdrawn,Balance,Balance Confirmed,Reason Type,Other Party Info,Linked Transaction ID,A/C No.
2,RBL25CWF1M,21-02-2023 09:59:14,21-02-2023 09:58:25,Faraja Term Loan Principal to Disbursement Tra...,Completed,7454603.13,NaN,12298224.7,True,Faraja Term Loan Principal to Disbursement,4018601 - Asante Financial Services 1,NaN,NaN


In [25]:
missing_rep.drop_duplicates(subset=['Receipt No.'])

,Receipt No.,Completion Time,Initiation Time,Details,Transaction Status,Paid In,Withdrawn,Balance,Balance Confirmed,Reason Type,Other Party Info,Linked Transaction ID,A/C No.
101,RBL94XA697,21-02-2023 01:00:43,21-02-2023 01:00:43,Loan Soko Principal Settlement,Completed,NaN,-2718998.58,652043.94,True,Term Loan Principal Settlement,4018601 - Asante Financial Services 1,NaN,NaN
102,RBL64XAICO,21-02-2023 01:00:43,21-02-2023 01:00:43,Asante Term Loan Revenue Settlement,Completed,NaN,-220603.96,111104.82,True,Asante Term Loan Revenue Settlement,4018601 - Asante Financial Services 1,NaN,NaN
103,RBL64XACAM,21-02-2023 01:00:43,21-02-2023 01:00:43,Asante Term Loan Revenue Settlement,Completed,NaN,-62.31,111042.51,True,Asante Term Loan Revenue Settlement,4018601 - Asante Financial Services 1,NaN,NaN
104,RBL64XA69E,21-02-2023 01:00:43,21-02-2023 01:00:43,Loan Soko Principal Settlement,Completed,NaN,-7340.00,644703.94,True,Term Loan Principal Settlement,4018601 - Asante Financial Services 1,NaN,NaN
105,RBL14XAAS1,21-02-2023 01:00:43,21-02-2023 01:00:43,Loan Soko Principal Settlement,Completed,NaN,-4728264.55,3371042.52,True,Term Loan Principal Settlement,4018601 - Asante Financial Services 1,NaN,NaN
106,RBL04X9ZIQ,21-02-2023 01:00:43,21-02-2023 01:00:43,Asante Term Loan Revenue Settlement,Completed,NaN,-312995.16,331708.78,True,Asante Term Loan Revenue Settlement,4018601 - Asante Financial Services 1,NaN,NaN


In [26]:
in_mifos = warehouse_hook.get_pandas_df(
    sql="""
    select * from bloomlive.transactions_dimension where receipt_number in %(refs)s""",
    parameters={'refs': tuple(missing_disb['Receipt No.'].tolist())}
)

[2023-02-23 08:10:57,331] {base.py:73} INFO - Using connection ID 'airflow_warehouse_user' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/airflow/providers/common/sql/hooks/sql.py:190 UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

In [27]:
missing_disb[~missing_disb['Receipt No.'].isin(in_mifos['receipt_number'].tolist())]

,Receipt No.,Completion Time,Initiation Time,Details,Transaction Status,Paid In,Withdrawn,Balance,Balance Confirmed,Reason Type,Other Party Info,Linked Transaction ID,A/C No.
2,RBL25CWF1M,21-02-2023 09:59:14,21-02-2023 09:58:25,Faraja Term Loan Principal to Disbursement Tra...,Completed,7454603.13,NaN,12298224.7,True,Faraja Term Loan Principal to Disbursement,4018601 - Asante Financial Services 1,NaN,NaN


In [28]:
in_mifos_reps = warehouse_hook.get_pandas_df(
    sql="""
    select * from bloomlive.transactions_dimension where receipt_number in %(refs)s""",
    parameters={'refs': tuple(missing_rep['Receipt No.'].tolist())}
)

[2023-02-23 08:11:12,103] {base.py:73} INFO - Using connection ID 'airflow_warehouse_user' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/airflow/providers/common/sql/hooks/sql.py:190 UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

In [29]:
missing_rep[~missing_rep['Receipt No.'].isin(in_mifos['receipt_number'].tolist())]

,Receipt No.,Completion Time,Initiation Time,Details,Transaction Status,Paid In,Withdrawn,Balance,Balance Confirmed,Reason Type,Other Party Info,Linked Transaction ID,A/C No.
101,RBL94XA697,21-02-2023 01:00:43,21-02-2023 01:00:43,Loan Soko Principal Settlement,Completed,NaN,-2718998.58,652043.94,True,Term Loan Principal Settlement,4018601 - Asante Financial Services 1,NaN,NaN
102,RBL64XAICO,21-02-2023 01:00:43,21-02-2023 01:00:43,Asante Term Loan Revenue Settlement,Completed,NaN,-220603.96,111104.82,True,Asante Term Loan Revenue Settlement,4018601 - Asante Financial Services 1,NaN,NaN
103,RBL64XACAM,21-02-2023 01:00:43,21-02-2023 01:00:43,Asante Term Loan Revenue Settlement,Completed,NaN,-62.31,111042.51,True,Asante Term Loan Revenue Settlement,4018601 - Asante Financial Services 1,NaN,NaN
104,RBL64XA69E,21-02-2023 01:00:43,21-02-2023 01:00:43,Loan Soko Principal Settlement,Completed,NaN,-7340.00,644703.94,True,Term Loan Principal Settlement,4018601 - Asante Financial Services 1,NaN,NaN
105,RBL14XAAS1,21-02-2023 01:00:43,21-02-2023 01:00:43,Loan Soko Principal Settlement,Completed,NaN,-4728264.55,3371042.52,True,Term Loan Principal Settlement,4018601 - Asante Financial Services 1,NaN,NaN
106,RBL04X9ZIQ,21-02-2023 01:00:43,21-02-2023 01:00:43,Asante Term Loan Revenue Settlement,Completed,NaN,-312995.16,331708.78,True,Asante Term Loan Revenue Settlement,4018601 - Asante Financial Services 1,NaN,NaN


In [29]:
warehouse_trxns = warehouse_hook.get_pandas_df(
    sql="""
        select * from bloomlive.transactions_dimension
        where transaction_date between '2023-01-01' and '2023-01-31'
    """
)

[2023-02-08 09:17:08,064] {base.py:73} INFO - Using connection ID 'airflow_warehouse_user' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/airflow/providers/common/sql/hooks/sql.py:190 UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

In [28]:
mifos_trxns = mifos_hook.get_pandas_df(
    sql="""
        select
            ml.id as loan_id, mpd.id as payment_id, mpd.account_number, mpt.value as payment_detail_id,
            mpd.receipt_number, mlt.id as transaction_id, mlt.office_id,
            mlt.is_reversed, mlt.transaction_type_enum, mlt.transaction_date, mlt.amount,
            mlt.principal_portion_derived, mlt.interest_portion_derived, mlt.fee_charges_portion_derived,
            mlt.submitted_on_date, mlt.created_date, mlt.appuser_id, mlt.outstanding_loan_balance_derived
        from `mifostenant-safaricom`.m_loan ml
        inner join `mifostenant-safaricom`.m_loan_transaction mlt on mlt.loan_id = ml.id
        inner join `mifostenant-safaricom`.m_payment_detail mpd on mpd.id = mlt.payment_detail_id
        left join `mifostenant-safaricom`.m_payment_type mpt on mpt.id = mpd.payment_type_id
        where mlt.transaction_type_enum in (1, 2) and mlt.is_reversed = 0 and date(mlt.transaction_date) between '2023-01-01' and '2023-01-31'
    """
)

[2023-02-08 09:16:52,279] {base.py:73} INFO - Using connection ID 'mifos_db' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/airflow/providers/common/sql/hooks/sql.py:190 UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

In [30]:
extra_trxns = warehouse_trxns[~warehouse_trxns['transaction_id'].isin(mifos_trxns['transaction_id'].tolist())]

# extra_trxns = mifos_trxns[~mifos_trxns['transaction_id'].isin(warehouse_trxns['transaction_id'].tolist())]

In [32]:
extra_trxns

,surrogate_id,mifos_loan_id,payment_id,account_number,payment_detail_id,receipt_number,transaction_id,is_reversed,transaction_type_enum,transaction_date,...,fee_charges_portion_derived,submitted_on_date,created_date,outstanding_loan_balance_derived,appuser_id,record_added_to_warehouse_on_timestamp,bloom_version,is_disbursement,is_repayment,office_id
6272,9490747,294577,822636,294577.0,MpesaRepayment,RA60TZEBRO,3663299,False,2,2023-01-06,...,178.08,2023-02-07,2023-01-06 17:35:32,27629.74,1,2023-02-08 08:29:32.592503,2,False,True,1
6450,9490925,294549,822873,294549.0,MpesaRepayment,RA66UAYFKC,3663286,False,2,2023-01-06,...,91.71,2023-02-07,2023-01-06 20:25:07,7844.37,1,2023-02-08 08:29:32.592503,2,False,True,1
7970,9492445,294819,824155,294819.0,MpesaRepayment,RA80Z166IE,3663369,False,2,2023-01-08,...,96.88,2023-02-07,2023-01-08 19:52:32,4627.14,1,2023-02-08 08:29:32.592503,2,False,True,1
8264,9492739,294530,824703,294530.0,MpesaRepayment,RA90ZM8YPQ,3663276,False,2,2023-01-09,...,49.97,2023-02-07,2023-01-09 08:00:02,2100.00,1,2023-02-08 08:29:32.592503,2,False,True,1
8299,9492774,295359,824812,295359.0,MpesaRepayment,RA93ZRKOVL,3663592,False,2,2023-01-09,...,67.67,2023-02-07,2023-01-09 09:20:17,6585.94,1,2023-02-08 08:29:32.592503,2,False,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22108,9507221,295523,844458,295523.0,MpesaRepayment,RAF1GVDNDZ,3656571,False,2,2023-01-15,...,NaN,2023-02-06,2023-01-26 09:55:55,7000.00,1,2023-02-08 08:29:51.797608,2,False,True,1
22109,9507222,296748,844459,296748.0,MpesaRepayment,RAF4GTDI1M,3663937,False,2,2023-01-15,...,6.27,2023-02-07,2023-01-26 09:56:10,113.16,1,2023-02-08 08:29:51.797608,2,False,True,1
22136,9507249,296098,844487,296098.0,MpesaRepayment,RAD7AXU6HV,3663843,False,2,2023-01-13,...,464.04,2023-02-07,2023-01-26 10:06:01,1073.27,1,2023-02-08 08:29:51.797608,2,False,True,1
22138,9507251,298725,844489,298725.0,MpesaRepayment,RAC78J16PD,3664377,False,2,2023-01-12,...,NaN,2023-02-07,2023-01-26 10:07:31,122.33,1,2023-02-08 08:29:51.797608,2,False,True,1


In [7]:
traced_transactions = mifos_hook.get_pandas_df(
    sql="""
        select
            mlt.*
        from `mifostenant-safaricom`.m_loan ml
        inner join `mifostenant-safaricom`.m_loan_transaction mlt on mlt.loan_id = ml.id
        inner join `mifostenant-safaricom`.m_payment_detail mpd on mpd.id = mlt.payment_detail_id
        left join `mifostenant-safaricom`.m_payment_type mpt on mpt.id = mpd.payment_type_id
        where mlt.transaction_type_enum in (1, 2) and date(mlt.transaction_date) -- between '2022-11-01' and '2022-11-30'
        and mlt.id in %(extra_trxns)s
    """,
    parameters={'extra_trxns': tuple(extra_trxns['transaction_id'].tolist())}
)

NameError: name 'extra_trxns' is not defined

In [37]:
traced_transactions

Column 'A' does not exist in the Series


In [7]:
existing_records = warehouse_hook.get_pandas_df(
    sql="""
        SELECT receipt_number FROM bloomlive.transactions_dimension
        WHERE bloom_version = 2;
    """
)

if len(existing_records) > 0:
    # extract only those records that are not already stored in the warehouse
    transactions = mifos_hook.get_pandas_df(
        sql=f"""
            select
                ml.id as loan_id, mpd.id as payment_id, mpd.account_number, mpt.value as payment_detail_id,
                mpd.receipt_number, mlt.id as transaction_id,
                mlt.is_reversed, mlt.transaction_type_enum, mlt.transaction_date, mlt.amount,
                mlt.principal_portion_derived, mlt.interest_portion_derived, mlt.fee_charges_portion_derived,
                mlt.submitted_on_date, mlt.created_date, mlt.appuser_id, mlt.outstanding_loan_balance_derived
            from `mifostenant-safaricom`.m_loan ml
            inner join `mifostenant-safaricom`.m_loan_transaction mlt on mlt.loan_id = ml.id
            inner join `mifostenant-safaricom`.m_payment_detail mpd on mpd.id = mlt.payment_detail_id
            left join `mifostenant-safaricom`.m_payment_type mpt on mpt.id = mpd.payment_type_id
            where mlt.transaction_type_enum in (1, 2) and mlt.is_reversed = 0 and mpd.receipt_number not in %(rns)s
        """,
        parameters={'rns': tuple(existing_records['receipt_number'].tolist())}
    )

In [8]:
extra_trxns2

,loan_id,payment_id,account_number,payment_detail_id,receipt_number,transaction_id,is_reversed,transaction_type_enum,transaction_date,amount,principal_portion_derived,interest_portion_derived,fee_charges_portion_derived,submitted_on_date,created_date,appuser_id,outstanding_loan_balance_derived
0,240795,730474,240795,MpesaRepayment,QKN2ISPA06,3037098,1,2,2022-11-23,5600.00,5600.00,NaN,NaN,2022-11-23,2022-11-23 15:41:54,1,46170.06
1,206190,619089,206190,MpesaRepayment,QIM1HE6WZ7,3066705,1,2,2022-09-22,8500.00,8500.00,NaN,NaN,2022-11-28,2022-10-06 08:21:48,1,86500.00
2,206190,619099,206190,MpesaRepayment,QIM5GZOVUT,3073085,1,2,2022-09-22,4400.00,2720.83,1679.17,NaN,2022-11-29,2022-10-06 08:22:28,1,83779.17
3,252468,739407,252468,MpesaRepayment,QKT5VPN9HP,3077984,1,2,2022-11-29,400.00,400.00,NaN,NaN,2022-11-29,2022-11-29 08:09:00,1,14509.68
4,153111,755917,153111,MpesaRepayment,QL93QZ9JR7,3165377,1,2,2022-12-09,5000.00,NaN,NaN,136.97,2022-12-12,2022-12-12 06:30:27,1,9700.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3480,291551,822044,291551,MpesaRepayment,RA65T1DHR3,3553349,1,2,2023-01-06,8497.08,7724.31,249.60,494.98,2023-01-24,2023-01-06 13:24:59,1,275.69
3481,291665,830178,291665,MpesaRepayment,RAC29OK5MM,3553371,1,2,2023-01-12,9290.97,8207.61,265.20,525.92,2023-01-24,2023-01-12 18:18:17,1,292.39
3482,291694,824013,291694,MpesaRepayment,RA89YS3TAF,3553376,1,2,2023-01-08,4673.39,4248.32,137.28,272.24,2023-01-24,2023-01-08 17:45:54,1,151.68
3483,295345,833392,295345,MpesaRepayment,RAG0JI0MQ0,3554222,1,2,2023-01-16,40000.00,38402.10,1560.00,16.80,2023-01-24,2023-01-16 19:47:09,1,597.90


In [10]:
extra_trxns3 = warehouse_hook.get_pandas_df(
    sql="""
        select * from bloomlive.transactions_dimension where transaction_id in %(ti)s
    """,
    parameters={'ti': tuple(extra_trxns['transaction_id'].tolist())}
)

In [12]:
extra_trxns3

In [59]:
mrged[mrged['transaction_id'] == 3477261]